In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [3]:
data = pd.read_excel('data_ford_price.xlsx')

In [4]:
data.isna().sum()

price              0
year               0
condition          0
cylinders          0
odometer           0
title_status       0
transmission       0
drive            391
size            1564
lat                0
long               0
weather          180
dtype: int64

In [5]:
data.describe()

,price,year,condition,cylinders,odometer,lat,long,weather
count,7017.000000,7017.000000,7017.000000,7017.000000,7.017000e+03,7017.000000,7017.000000,6837.000000
mean,15121.549523,2007.869745,2.598689,7.374662,1.199787e+05,39.550144,-91.903404,52.142899
std,11765.423119,6.975329,0.703662,0.942928,8.992216e+04,5.745409,14.030710,7.954830
min,1.000000,1957.000000,0.000000,3.000000,0.000000e+00,-2.508807,-151.055832,29.000000
25%,5995.000000,2004.000000,2.000000,6.000000,7.328500e+04,35.661076,-95.937145,45.000000
50%,12750.000000,2010.000000,3.000000,8.000000,1.180000e+05,40.335245,-88.168416,51.000000
75%,21995.000000,2013.000000,3.000000,8.000000,1.578040e+05,43.582100,-82.706300,59.000000
max,299500.000000,2018.000000,5.000000,10.000000,2.490000e+06,77.617682,-5.377999,71.000000


In [6]:
data['size'].value_counts()

full-size      4570
mid-size        867
compact          15
sub-compact       1
Name: size, dtype: int64

In [7]:
test_data = data[data['size'].isnull()]
test_data

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN
8,18750,2011,2,6,77000,clean,automatic,rwd,NaN,33.698900,-84.745400,59.0
12,1700,2001,1,8,275000,clean,automatic,4wd,NaN,44.549400,-90.611200,45.0
25,35999,2016,3,8,25814,clean,automatic,4wd,NaN,43.852883,-88.855927,45.0
33,10000,2005,2,8,148000,clean,automatic,4wd,NaN,61.153500,-149.828900,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6983,3500,2004,1,8,179945,clean,automatic,rwd,NaN,40.010899,-74.131575,52.0
6987,22500,2014,3,6,91383,clean,automatic,4wd,NaN,43.223605,-76.802231,53.0
6991,6600,2010,2,8,101215,clean,automatic,rwd,NaN,43.098946,-76.097853,53.0
7007,28500,2014,3,8,40000,clean,automatic,4wd,NaN,36.562900,-82.653200,59.0


In [8]:
data.dropna(inplace=True)

In [9]:
y_train = data['size']

In [10]:
X_train = data.drop(['size', 'weather', 'drive'], axis=1)

In [11]:
X_test = test_data.drop(['size', 'weather', 'drive'], axis=1)

In [12]:
categorial_cols = ['cylinders', 'title_status', 'transmission']

In [13]:
def encode_cat_features(columns_to_change, X_train, X_test, y_train):
  one_hot_encoder = OneHotEncoder()
  X_train_onehot = one_hot_encoder.fit_transform(X_train[columns_to_change]).toarray()
  X_test_onehot = one_hot_encoder.transform(X_test[columns_to_change]).toarray()

  columns = one_hot_encoder.get_feature_names(columns_to_change)
  
  X_train_onehot_df = pd.DataFrame(X_train_onehot, columns=columns)
  X_test_onehot_df = pd.DataFrame(X_test_onehot, columns=columns)

  X_train = X_train.reset_index().drop(['index'], axis = 1)
  X_test = X_test.reset_index().drop(['index'], axis = 1)
  y_train = y_train.reset_index().drop(['index'], axis = 1)

  X_train_new = pd.concat([X_train, X_train_onehot_df], axis=1)
  X_test_new = pd.concat([X_test, X_test_onehot_df], axis=1)
  
  X_train_new = X_train_new.drop(columns=columns_to_change)
  X_test_new = X_test_new.drop(columns=columns_to_change)

  return X_train_new, X_test_new

In [14]:
X_train_new, X_test_new = encode_cat_features(categorial_cols, X_train, X_test, y_train)

/home/egor/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train_new, y_train)
 
y_pred = log_reg.predict(X_test_new)
y_pred

/home/egor/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array(['full-size', 'full-size', 'full-size', ..., 'full-size',
       'full-size', 'full-size'], dtype=object)

In [16]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5202 entries, 0 to 7016
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         5202 non-null   int64  
 1   year          5202 non-null   int64  
 2   condition     5202 non-null   int64  
 3   cylinders     5202 non-null   int64  
 4   odometer      5202 non-null   int64  
 5   title_status  5202 non-null   object 
 6   transmission  5202 non-null   object 
 7   lat           5202 non-null   float64
 8   long          5202 non-null   float64
dtypes: float64(2), int64(5), object(2)
memory usage: 406.4+ KB


In [17]:
dt = pd.read_csv('data/adult.csv')
dt

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32556,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32557,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32558,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
